<a href="https://colab.research.google.com/github/pawanacharya1979/CS599_DL/blob/main/Lab3_Code(CS_599).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np

# -----------------------------------------------------------
# 1. Custom Normalization Functions
# -----------------------------------------------------------
def custom_batch_norm(x, gamma, beta, epsilon=1e-5):
    """
    Custom Batch Normalization.
    Computes the mean and variance over the mini-batch (and spatial dimensions for CNNs)
    then normalizes and rescales the input.

    x: Input tensor.
    gamma: Learnable scale parameter.
    beta: Learnable shift parameter.
    epsilon: Small constant for numerical stability.
    """
    axes = list(range(len(x.shape) - 1))  # Normalize over batch, height, and width dimensions
    batch_mean = tf.reduce_mean(x, axis=axes, keepdims=True)
    batch_variance = tf.reduce_mean(tf.square(x - batch_mean), axis=axes, keepdims=True)
    x_norm = (x - batch_mean) / tf.sqrt(batch_variance + epsilon)
    return gamma * x_norm + beta

def custom_weight_norm(v, g, axis=None, epsilon=1e-5):
    """
    Custom Weight Normalization.
    Reparameterizes a weight vector v using a scalar g such that:
         w = (g / ||v||) * v,
    where ||v|| is the Euclidean norm of v.

    v: Weight vector (or tensor).
    g: Learnable scalar (or tensor, broadcastable to v) controlling the magnitude.
    axis: The axis (or axes) along which to compute the norm.
    epsilon: Small constant for numerical stability.
    """
    v_norm = tf.sqrt(tf.reduce_sum(tf.square(v), axis=axis, keepdims=True) + epsilon)
    return (g / v_norm) * v

def custom_layer_norm(x, gamma, beta, epsilon=1e-5):
    """
    Custom Layer Normalization.
    Normalizes the input tensor x across the feature dimension for each sample.

    x: Input tensor with shape [..., features].
    gamma: Learnable scale parameter (broadcastable to x).
    beta: Learnable shift parameter (broadcastable to x).
    epsilon: Small constant for numerical stability.
    """
    mean = tf.reduce_mean(x, axis=-1, keepdims=True)
    variance = tf.reduce_mean(tf.square(x - mean), axis=-1, keepdims=True)
    x_norm = (x - mean) / tf.sqrt(variance + epsilon)
    return gamma * x_norm + beta

# -----------------------------------------------------------
# 2. CNN Model with Normalization Options
# -----------------------------------------------------------
class CustomCNN(tf.keras.Model):
    def __init__(self, num_classes=10, norm_type='none'):
        """
        A CNN model that applies the specified normalization in the forward pass.

        norm_type: 'batch', 'layer', 'weight', or 'none'
        """
        super(CustomCNN, self).__init__()
        self.norm_type = norm_type

        # For standard convolution (if not using weight normalization)
        if self.norm_type != 'weight':
            self.conv1 = tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', use_bias=True)
        else:
            # For weight normalization, initialize kernel components separately.
            self.kernel_shape = (3, 3, 1, 32)
            self.v = tf.Variable(tf.random.normal(self.kernel_shape, stddev=0.1), trainable=True)
            self.g = tf.Variable(tf.ones((1,)), trainable=True)
            self.conv1_bias = tf.Variable(tf.zeros([32]), trainable=True)

        # For Batch Normalization: create learnable gamma and beta parameters.
        if self.norm_type == 'batch':
            self.gamma_bn = tf.Variable(tf.ones([1, 1, 1, 32]), trainable=True)
            self.beta_bn  = tf.Variable(tf.zeros([1, 1, 1, 32]), trainable=True)
        # For Layer Normalization: create gamma and beta per channel.
        elif self.norm_type == 'layer':
            self.gamma_ln = tf.Variable(tf.ones([32]), trainable=True)
            self.beta_ln  = tf.Variable(tf.zeros([32]), trainable=True)

        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes)

    def call(self, x, training=False):
        # Convolution operation.
        if self.norm_type == 'weight':
            kernel = custom_weight_norm(self.v, self.g, axis=[0, 1, 2])
            conv = tf.nn.conv2d(x, kernel, strides=1, padding='SAME') + self.conv1_bias
        else:
            conv = self.conv1(x)

        # Apply normalization based on the chosen norm_type.
        if self.norm_type == 'batch':
            conv = custom_batch_norm(conv, self.gamma_bn, self.beta_bn)
        elif self.norm_type == 'layer':
            # Reshape to merge spatial dimensions for layer normalization.
            shape = tf.shape(conv)
            conv_reshaped = tf.reshape(conv, [shape[0], -1, conv.shape[-1]])
            conv_norm = custom_layer_norm(conv_reshaped, self.gamma_ln, self.beta_ln)
            conv = tf.reshape(conv_norm, shape)
        # If norm_type is 'none', nothing extra is done.

        # Activation and pooling.
        x_act = tf.nn.relu(conv)
        x_pool = self.pool1(x_act)
        x_flat = self.flatten(x_pool)
        logits = self.fc(x_flat)
        return logits

# -----------------------------------------------------------
# 3. Training Setup: Loss, Optimizer, and Training Step
# -----------------------------------------------------------
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss = loss_object(labels, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# -----------------------------------------------------------
# 4. Main Function: Data Preparation, Training, Evaluation, and Comparison
# -----------------------------------------------------------
def main():
    # Data Preparation: Load Fashion MNIST dataset.
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = np.expand_dims(x_train.astype(np.float32) / 255.0, -1)
    x_test = np.expand_dims(x_test.astype(np.float32) / 255.0, -1)

    batch_size = 64
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
    num_epochs = 5

    # Instantiate a model.
    # Change norm_type to one of 'batch', 'layer', 'weight', or 'none' as desired.
    model = CustomCNN(norm_type='layer')

    # Build the model by calling it on a dummy input.
    dummy_input = tf.random.normal([1, 28, 28, 1])
    _ = model(dummy_input, training=True)

    # Training Loop.
    for epoch in range(num_epochs):
        total_loss = 0.0
        steps = 0
        for images, labels in train_dataset:
            loss = train_step(model, images, labels)
            total_loss += loss
            steps += 1
        print(f"Epoch {epoch+1} (Layer Norm): Loss = {total_loss/steps:.4f}")

    # Evaluation on the Test Data.
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    for images, labels in test_dataset:
        logits = model(images, training=False)
        test_accuracy.update_state(labels, logits)
    print(f"Test Accuracy (Layer Norm): {test_accuracy.result().numpy()*100:.2f}%")

    # -----------------------------------------------------------
    # 5. Comparison with TensorFlow Built-In Normalization Functions
    # -----------------------------------------------------------
    sample_input = tf.random.normal([32, 28, 28, 32])
    custom_bn_output = custom_batch_norm(sample_input,
                                         gamma=tf.ones([1,1,1,32]),
                                         beta=tf.zeros([1,1,1,32]))
    bn_layer = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-5)
    tf_bn_output = bn_layer(sample_input, training=True)
    difference = tf.reduce_mean(tf.abs(custom_bn_output - tf_bn_output))
    print(f"Mean absolute difference between custom BN and TF BN: {difference.numpy():.6f}")

if __name__ == '__main__':
    main()


Epoch 1 (Layer Norm): Loss = 0.4115
Epoch 2 (Layer Norm): Loss = 0.3032
Epoch 3 (Layer Norm): Loss = 0.2819
Epoch 4 (Layer Norm): Loss = 0.2648
Epoch 5 (Layer Norm): Loss = 0.2533
Test Accuracy (Layer Norm): 89.21%
Mean absolute difference between custom BN and TF BN: 0.000000
